### Imports

In [1]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 172 µs (started: 2023-06-11 15:04:55 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = params["method"]
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t\t", option)

if option:
    
    subgroups_id_op = {}
    for group in groups_id:
        subgroups_id_op[group] = [option]
else:
    subgroups_id_op = subgroups_id
print("Subgroups id op:", subgroups_id_op)



Exp:		 exp2
Method:		 vgae
Dimension:	 3
Groups id:	 ['WT', 'zwf1^', 'pck1^']
Subgroups id:	 {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
Option:		 str
Subgroups id op: {'WT': ['str'], 'zwf1^': ['str'], 'pck1^': ['str']}
time: 2.78 ms (started: 2023-06-11 15:04:56 -05:00)


### Edge embeddings

In [3]:
# get edges embeddings

edge_embeddings_global(exp, method, groups_id, subgroups_id_op)

100%|██████████| 3/3 [47:25<00:00, 948.49s/it] 

time: 47min 25s (started: 2023-06-11 15:04:57 -05:00)


In [4]:
df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, groups_id[0], subgroups_id_op[groups_id[0]][0]), index_col=[0, 1])
df_edge_embeddings.head()

0         1         2
0 1  0.277892  1.972417  0.036018
  2  0.260216  3.282508  0.556048
  3  1.831183  2.303905  0.017388
  4  0.198650  3.902381  0.028556
  5  0.000298  6.091694  0.118930

time: 4.56 s (started: 2023-06-11 15:52:22 -05:00)


### Concat edge embeddings

In [5]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    k = 0
    for subgroup in tqdm(subgroups_id_op[group]):
        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, group, subgroup), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)
        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
        k += 1
    
    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index=True)

100%|██████████| 3/3 [03:11<00:00, 63.73s/it]

time: 3min 11s (started: 2023-06-11 15:52:27 -05:00)


In [6]:
df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, groups_id[0], option), index_col=[0, 1])
df_edge_embeddings_concat.head()

0         1         2  subgroup
0 1  0.277892  1.972417  0.036018         0
  2  0.260216  3.282508  0.556048         0
  3  1.831183  2.303905  0.017388         0
  4  0.198650  3.902381  0.028556         0
  5  0.000298  6.091694  0.118930         0

time: 4.56 s (started: 2023-06-11 15:55:40 -05:00)


In [7]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

100%|██████████| 3/3 [14:09<00:00, 283.31s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 14min 9s (started: 2023-06-11 15:55:44 -05:00)


### Outliers detection

In [8]:
# Outlier detection (HDBSCAN)

""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

' df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])\n\nX_train = df_edge_embeddings_concat.iloc[:, :-1]\nclusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)\n\nthreshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)\noutliers = np.where(clusterer.outlier_scores_ > threshold)[0]\nprint(len(outliers))\noutliers\n\ninliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)\nprint(len(inliers))\ninliers '

time: 4.69 ms (started: 2023-06-11 16:09:55 -05:00)


In [9]:
# outlier detection (ECOD)

dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:56<00:00, 18.94s/it]

time: 56.8 s (started: 2023-06-11 16:09:55 -05:00)


In [10]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
0 1    0.277892  1.972417  0.036018         0
  2    0.260216  3.282508  0.556048         0
  8    0.838389  0.221567  0.158415         0
  9    0.520851  0.000743  0.062668         0
1 131  0.016451  0.177310  0.709647         0

time: 13.1 ms (started: 2023-06-11 16:10:52 -05:00)


In [11]:
# plot outliers/inliers

for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

  0%|          | 0/3 [00:00<?, ?it/s]

Total: 12232242
Num. of inliers: 11009017
Num. of inliers: 1223225


 33%|███▎      | 1/3 [04:48<09:36, 288.10s/it]

Total: 17089035
Num. of inliers: 15380131
Num. of inliers: 1708904


 67%|██████▋   | 2/3 [11:21<05:49, 349.96s/it]

...

100%|██████████| 3/3 [13:39<00:00, 273.11s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 13min 39s (started: 2023-06-11 16:10:53 -05:00)


###  Filter common edges

In [12]:
print(subgroups_id)
print(subgroups_id_op)

{'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
{'WT': ['str'], 'zwf1^': ['str'], 'pck1^': ['str']}
time: 1.08 ms (started: 2023-06-11 16:24:33 -05:00)


In [13]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id_op[group]:
        df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup),
                               dtype={"id": "string"})
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []
    
    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[subgroups_id_op[group][row[-1]]]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)

11009017it [05:12, 35178.56it/s]it/s]
15380131it [07:19, 34960.25it/s]3, 321.64s/it]
5439862it [02:32, 35625.48it/s]38, 398.39s/it]
100%|██████████| 3/3 [15:32<00:00, 310.77s/it]

time: 15min 32s (started: 2023-06-11 16:24:35 -05:00)


In [14]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
A59.0049  A274.0123  0.277892  1.972417  0.036018         0
          A277.0867  0.260216  3.282508  0.556048         0
          A641.1223  0.838389  0.221567  0.158415         0
          A650.2145  0.520851  0.000743  0.062668         0
A274.0123 A191.046   0.016451  0.177310  0.709647         0

time: 13 ms (started: 2023-06-11 16:40:08 -05:00)


In [15]:
# format id
if option:
    for group in tqdm(groups_id):
        # format
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        list_index = []

        for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
            list_index.append((row[0][0][1:], row[0][1][1:]))
        
        # set new index
        df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
        df_edge_embeddings_concat_filter

11009017it [00:10, 1095247.02it/s]/s]
15380131it [00:14, 1080717.75it/s] 22.06s/it]
5439862it [00:04, 1137092.26it/s], 25.83s/it]
100%|██████████| 3/3 [01:00<00:00, 20.18s/it]

time: 1min (started: 2023-06-11 16:40:08 -05:00)


In [16]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
59.0049  274.0123  0.277892  1.972417  0.036018         0
         277.0867  0.260216  3.282508  0.556048         0
         641.1223  0.838389  0.221567  0.158415         0
         650.2145  0.520851  0.000743  0.062668         0
274.0123 191.046   0.016451  0.177310  0.709647         0

time: 11.9 ms (started: 2023-06-11 16:41:10 -05:00)


In [17]:
# filter diferente edges

if option:
    for group in tqdm(groups_id):
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter.index.get_level_values(0) != df_edge_embeddings_concat_filter.index.get_level_values(1)]
        dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:03<00:00,  1.19s/it]

time: 3.57 s (started: 2023-06-11 16:41:10 -05:00)


In [18]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
print(df_edge_embeddings_concat_filter.shape)
df_edge_embeddings_concat_filter.head()

(11009015, 4)


0         1         2  subgroup
59.0049  274.0123  0.277892  1.972417  0.036018         0
         277.0867  0.260216  3.282508  0.556048         0
         641.1223  0.838389  0.221567  0.158415         0
         650.2145  0.520851  0.000743  0.062668         0
274.0123 191.046   0.016451  0.177310  0.709647         0

time: 12.3 ms (started: 2023-06-11 16:41:14 -05:00)


In [19]:
# count edges and filter by count

dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # count
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["level_0", "level_1"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]

    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:15<00:00,  5.00s/it]

time: 15 s (started: 2023-06-11 16:41:14 -05:00)


In [20]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,378.089,430.1283
1,688.4522,865.5704
2,857.5408,864.5667
3,817.5496,817.4706
4,595.3766,818.6501


time: 7.24 ms (started: 2023-06-11 16:41:29 -05:00)


In [21]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26670 entries, 0 to 26669
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  26670 non-null  object
 1   target  26670 non-null  object
dtypes: object(2)
memory usage: 625.1+ KB
time: 21.4 ms (started: 2023-06-11 16:41:30 -05:00)


In [22]:
# change data type
for group in tqdm(groups_id):
    df_edges_filter = dict_df_edges_filter[group]
    df_edges_filter[["source", "target"]] = df_edges_filter[["source", "target"]].astype("string")

100%|██████████| 3/3 [00:00<00:00, 30.52it/s]

time: 103 ms (started: 2023-06-11 16:41:30 -05:00)


In [23]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,378.089,430.1283
1,688.4522,865.5704
2,857.5408,864.5667
3,817.5496,817.4706
4,595.3766,818.6501


time: 7.3 ms (started: 2023-06-11 16:41:30 -05:00)


In [24]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26670 entries, 0 to 26669
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  26670 non-null  string
 1   target  26670 non-null  string
dtypes: string(2)
memory usage: 625.1 KB
time: 14.3 ms (started: 2023-06-11 16:41:31 -05:00)


In [25]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

100%|██████████| 3/3 [01:11<00:00, 23.94s/it]


,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
19767,102.0562,96.9761,0.613406,0.573434,0.587553,0.830119,0.845229
26017,109.0407,127.0513,0.791174,0.792589,0.782065,0.917810,0.870420
25918,109.0407,131.0824,0.808212,0.663848,0.719276,0.857543,0.913011
26284,109.0407,144.0458,0.688714,0.710822,0.875062,0.922401,0.783513
24868,109.0407,272.0889,0.733025,0.594636,0.751374,0.822002,0.867299


time: 1min 12s (started: 2023-06-11 16:41:31 -05:00)


In [26]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
19767,102.0562,96.9761,0.613406,0.573434,0.587553,0.830119,0.845229
26017,109.0407,127.0513,0.791174,0.792589,0.782065,0.917810,0.870420
25918,109.0407,131.0824,0.808212,0.663848,0.719276,0.857543,0.913011
26284,109.0407,144.0458,0.688714,0.710822,0.875062,0.922401,0.783513
24868,109.0407,272.0889,0.733025,0.594636,0.751374,0.822002,0.867299


time: 15.6 ms (started: 2023-06-11 16:42:44 -05:00)


### Filter by STD and average weight

In [27]:
dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

100%|██████████| 3/3 [00:15<00:00,  5.21s/it]


,source,target,weight
0,102.0562,96.9761,0.689948
1,109.0407,127.0513,0.830812
2,109.0407,131.0824,0.792378
3,109.0407,144.0458,0.796102
4,109.0407,272.0889,0.753667


<Figure size 640x480 with 0 Axes>

time: 15.6 s (started: 2023-06-11 16:42:46 -05:00)


In [28]:
df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, groups_id[0], option),
                              dtype={"source": "string", "target": "string"})
df_common_edges.head()

,source,target,weight
0,102.0562,96.9761,0.689948
1,109.0407,127.0513,0.830812
2,109.0407,131.0824,0.792378
3,109.0407,144.0458,0.796102
4,109.0407,272.0889,0.753667


time: 30.2 ms (started: 2023-06-11 16:43:01 -05:00)


In [29]:
# show details

for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, option))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

 33%|███▎      | 1/3 [00:00<00:01,  1.89it/s]

Group: WT
Num. nodes: 2276
Num. edges: 26449



 67%|██████▋   | 2/3 [00:01<00:00,  1.22it/s]

Group: zwf1^
Num. nodes: 4890
Num. edges: 504767



100%|██████████| 3/3 [00:02<00:00,  1.22it/s]

Group: pck1^
Num. nodes: 5259
Num. edges: 439912

time: 2.46 s (started: 2023-06-11 16:43:02 -05:00)
